<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/model/Supervised%20Learning/experiments_sts16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers
from datasets import load_dataset
dataset = load_dataset("mteb/sts16-sts")

In [17]:
import sys
import os
import gzip
import csv

In [11]:
from transformers import AutoTokenizer

In [33]:
#sts_dataset_path = "https://datasets-server.huggingface.co/first-rows?dataset=mteb%2Fsts16-sts&config=mteb--sts16-sts"
sts_dataset_path= "https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/STS2017-extended.zip"   

In [ ]:
https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/STS2017-extended.zip    

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [37]:
task ="sts16"
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util

In [38]:
if not os.path.exists(sts_dataset_path):
    util.http_get('https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/STS2017-extended.zip', sts_dataset_path)

  0%|          | 0.00/96.3k [00:00<?, ?B/s]

In [39]:
train_samples = []
dev_samples = []
test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

        if row['split'] == 'dev':
            dev_samples.append(inp_example)
        elif row['split'] == 'test':
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

OSError: ignored

In [27]:
task_to_keys = {
    "sts16": ("sentence1", "sentence2"),
}

In [28]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

KeyError: ignored

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

In [ ]:
train_samples = []
dev_samples = []
test_samples = []
#with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
#    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
for row in dataset:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

        if row['split'] == 'dev':
            dev_samples.append(inp_example)
        elif row['split'] == 'test':
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)


In [12]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)